In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
url = "https://my.pitt.edu"
email = '***'
passw = '***'

In [3]:
ids = pd.read_csv('/users/madke/documents/SBDL_ids.csv')

In [59]:
def loginPitt(email = email, passw = passw):
    '''
    INPUT: user credentials
    
    OUTPUT: driver
    
    logs in to Pitt
    
    '''
    driver = webdriver.Chrome()
    driver.get(url)

    menu_btn = driver.find_element_by_id("headerMenuDropdown")
    menu_btn.click()

    signin_btn = driver.find_element_by_xpath("/html/body/div[1]/div[4]/section[1]/div/nav/div/div[2]/ul/li[2]/ul/li[1]/a")
    signin_btn.click()

    username = driver.find_element_by_xpath("/html/body/div/div/div[1]/div/form/div[1]/input")
    username.send_keys(email)

    password =  driver.find_element_by_xpath("/html/body/div/div/div[1]/div/form/div[2]/input")
    password.send_keys(passw)

    submit_btn = driver.find_element_by_xpath("/html/body/div/div/div[1]/div/form/button")
    submit_btn.click()
    
    return(driver)

CALL SELF FOR SECURITY

In [7]:
def openOutlook(driver):
    '''
    INPUT:driver
    
    OUTPUT: driver
    
    '''
    email_btn = driver.find_element_by_xpath("/html/body/div[1]/div[5]/div[3]/div/div[2]/div[4]/div[3]/div[1]/div[1]/div/a")
    email_btn.click()
    driver.switch_to.window(driver.window_handles[1])
    email_search_activate = driver.find_element_by_id("searchBoxId-Mail")
    email_search_activate.click()
    email_search = driver.find_element_by_xpath("/html/body/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div/div/div[1]/div[2]/div/div/div/div/div[1]/div/div[2]/div/input")
    search_btn = driver.find_element_by_xpath("/html/body/div[2]/div/div[1]/div/div[1]/div[2]/div/div/div/div/div[1]/div[2]/div/div/div/div/div[1]/button")
    
    return(driver)

In [45]:
def return_name(email, driver):
    '''
    takes an email and gets name
    '''
    email_search.send_keys(email)
    email_search_activate.click()
    time.sleep(2)
    try:
        results = driver.find_element_by_id("searchSuggestionsCallout")
        soup = BeautifulSoup(results.get_attribute('innerHTML'), 'html.parser')
        name = soup.find("div", {"id":"searchSuggestion-0" })['aria-label']
    
        email_search.send_keys(Keys.CONTROL + "a")
        email_search.send_keys(Keys.DELETE)
    
        return(name, driver)
    except:
        email_search.send_keys(Keys.CONTROL + "a")
        email_search.send_keys(Keys.DELETE)
        return('NONE', driver)
    

In [ ]:
def full_path_to_names(ids):
    '''
    INPUT:
    
    OUTPUT:
    '''
    emails = ids.iloc['email'].values
    driver = loginPitt()
    print('PLEASE CONFIRM IN BROWSER')
    time.sleep(30)
    driver = openOutlook(driver)
    
    name_ls = []
    for i in emails:
        name = return_name(i, driver)[0]
        print(name)
        name_ls.append(name)
        
    emails2names = pd.DataFrame({
        'email': emails, 
        'name': names,
        'credit': ids['credit'],
        'action': ids['action']
    })
    
    return(emails2names)

In [58]:
for_sona = emails2names[emails2names['action'].isnull()]
for_sona.to_csv('/users/madke/documents/111020for_sona.csv')